# Imports

In [ ]:
%matplotlib inline
import os, sys
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # for debugging GPU stuff
import time, random

import torch
import torch.optim as optim
from tensorboardX import SummaryWriter
import numpy as np

from importlib import reload

In [ ]:
# Config file
cfg_file = '/home/chrisxie/local_installations/PointGroup/config/pointgroup_TOD.yaml'
from util.config import get_parser_notebook
get_parser_notebook(cfg_file=cfg_file, pretrain_path=None)

from util.config import cfg
from util.log import logger
import util.utils as utils

# Training function definitions

In [ ]:
def init():
    # copy important files to backup
    backup_dir = os.path.join(cfg.exp_path, 'backup_files')
    os.makedirs(backup_dir, exist_ok=True)
    os.system('cp train.py {}'.format(backup_dir))
    os.system('cp {} {}'.format(cfg.model_dir, backup_dir))
    os.system('cp {} {}'.format(cfg.dataset_dir, backup_dir))
    os.system('cp {} {}'.format(cfg.config, backup_dir))

    # log the config
    logger.info(cfg)

    # summary writer
    global writer
    writer = SummaryWriter(cfg.exp_path)

    # random seed
    random.seed(cfg.manual_seed)
    np.random.seed(cfg.manual_seed)
    torch.manual_seed(cfg.manual_seed)
    torch.cuda.manual_seed_all(cfg.manual_seed)

In [ ]:
def train_epoch(train_loader, model, model_fn, optimizer, epoch):
    iter_time = utils.AverageMeter()
    data_time = utils.AverageMeter()
    am_dict = {}

    current_iter = (epoch - 1) * len(train_loader)
    
    model.train()
    start_epoch = time.time()
    end = time.time()
    for i, batch in enumerate(train_loader):
        data_time.update(time.time() - end)
        torch.cuda.empty_cache()

        if current_iter >= cfg.max_iters:
            break
        
        ##### adjust learning rate
        utils.step_learning_rate(optimizer, cfg.lr, epoch - 1, cfg.step_epoch, cfg.multiplier)

        ##### prepare input and forward
        loss, _, visual_dict, meter_dict = model_fn(batch, model, current_iter)

        ##### meter_dict
        for k, v in meter_dict.items():
            if k not in am_dict.keys():
                am_dict[k] = utils.AverageMeter()
            am_dict[k].update(v[0], v[1])

        ##### backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ##### time and print
        current_iter += 1
        remain_iter = cfg.max_iters - current_iter

        iter_time.update(time.time() - end)
        end = time.time()

        remain_time = remain_iter * iter_time.avg
        t_m, t_s = divmod(remain_time, 60)
        t_h, t_m = divmod(t_m, 60)
        remain_time = '{:02d}:{:02d}:{:02d}'.format(int(t_h), int(t_m), int(t_s))

        sys.stdout.write(
            "epoch: {}/{} iter: {}/{} loss: {:.4f}({:.4f}) data_time: {:.2f}({:.2f}) iter_time: {:.2f}({:.2f}) remain_time: {remain_time}\n".format
            (epoch, cfg.max_epochs, i + 1, len(train_loader), am_dict['loss'].val, am_dict['loss'].avg,
             data_time.val, data_time.avg, iter_time.val, iter_time.avg, remain_time=remain_time))
        if (i == len(train_loader) - 1): print()

    logger.info("epoch: {}/{}, train loss: {:.4f}, time: {}s".format(epoch, cfg.max_epochs, am_dict['loss'].avg, time.time() - start_epoch))

    utils.checkpoint_save(model, cfg.exp_path, cfg.config.split('/')[-1][:-5], epoch, cfg.save_freq, use_cuda)

    for k in am_dict.keys():
        if k in visual_dict.keys():
            writer.add_scalar(k+'_train', am_dict[k].avg, epoch)

# Training script

In [ ]:
##### init
init()

In [ ]:
##### get model version and data version
exp_name = cfg.config.split('/')[-1][:-5]
print(exp_name)
model_name = exp_name.split('_')[0]
print(model_name)
data_name = exp_name.split('_')[-1]
print(data_name)

In [ ]:
##### model
logger.info('=> creating model ...')
if model_name == 'pointgroup':
    from model.pointgroup.pointgroup import PointGroup as Network
    from model.pointgroup.pointgroup import model_fn_decorator
else:
    print("Error: no model - " + model_name)
    exit(0)
model = Network(cfg)

use_cuda = torch.cuda.is_available()
logger.info('cuda available: {}'.format(use_cuda))
assert use_cuda
model = model.cuda()

# logger.info(model)
logger.info('#classifier parameters: {}'.format(sum([x.nelement() for x in model.parameters()])))

In [ ]:
##### optimizer
logger.info('=> creating optimizer ...')
if cfg.optim == 'Adam':
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=cfg.lr)
elif cfg.optim == 'SGD':
    optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=cfg.lr, momentum=cfg.momentum, weight_decay=cfg.weight_decay)

In [ ]:
##### model_fn (criterion)
model_fn = model_fn_decorator()

In [ ]:
##### dataset
if cfg.dataset == 'TOD':
    if data_name == 'TOD':
        import data.TOD
        data.TOD = reload(data.TOD)
        dataset = data.TOD.Dataset()
        dataset.trainLoader()
    else:
        print("Error: no data loader - " + data_name)
        exit(0)

In [ ]:
##### resume
start_epoch = utils.checkpoint_restore(model,
                                       cfg.exp_path,
                                       cfg.config.split('/')[-1][:-5],
                                       use_cuda)
# resume from the latest epoch, or specify the epoch to restore
print(f'Start epoch: {start_epoch}')

In [ ]:
##### train and val
for epoch in range(start_epoch, cfg.max_epochs + 1):
    train_epoch(dataset.train_data_loader, model, model_fn, optimizer, epoch)

# Calculate statistics of dataset

In [ ]:
from tqdm.notebook import tqdm

distances = []

num_exs = 1000
batch_size = 5
for i in tqdm(range(0, num_exs, batch_size)):
    temp = dataset.train_collate_fn([i])
    fg_mask = temp['labels'] >= 0
    distances = distances + np.linalg.norm(temp['locs_float'][fg_mask] -
                                           temp['instance_info'][fg_mask][:,:3], axis=-1).tolist()

In [ ]:
np.quantile(distances, 0.5)

# Random Testing

In [ ]:
temp = dataset.train_collate_fn([0,1])

In [ ]:
temp.keys()

In [ ]:
xyz = temp['locs'].numpy().copy()[:,1:]
xyz -= xyz.min(0)

xyz_offset = xyz.copy()
valid_idxs = (xyz_offset.min(1) >= 0)
assert valid_idxs.sum() == xyz.shape[0]

full_scale = np.array([cfg.full_scale[1]] * 3)
room_range = xyz.max(0) - xyz.min(0)
while (valid_idxs.sum() > 250000):
    offset = np.clip(full_scale - room_range + 0.001, None, 0) * np.random.rand(3)
    xyz_offset = xyz + offset
    valid_idxs = (xyz_offset.min(1) >= 0) * ((xyz_offset < full_scale).sum(1) == 3)
    full_scale[:2] -= 32

xyz_offset, valid_idxs

In [ ]:
np.count_nonzero(valid_idxs)

In [ ]:
temp['v2p_map'][0]

In [ ]:
temp['v2p_map'].shape

In [ ]:
temp['p2v_map'].max()

In [ ]:
print(temp['v2p_map'].shape, temp['v2p_map'].numpy().size)

In [ ]:
temp['locs'].max()

In [ ]:
np.unique(temp['labels'].numpy())

In [ ]:
np.unique(temp['instance_labels'].numpy())

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(1, figsize=(10,5))

plt.subplot(1,2,1)
temp_ = temp['labels'].numpy().copy().reshape(480,640)
# temp_[temp_ == -100] = 0
plt.imshow(temp_)
plt.title('Seg Labels')

plt.subplot(1,2,2)
temp_ = temp['instance_labels'].numpy().copy().reshape(480,640) + 1
temp_[temp_ == -100+1] = 0
plt.imshow(temp_)
plt.title('Instance Labels')

In [ ]:
temp['locs'].shape

In [ ]:
temp['voxel_locs'].shape

In [ ]:
temp['locs_float'].shape

In [ ]:
temp['feats'].shape

In [ ]:
torch.is_tensor(temp['feats'])

In [ ]:
for key in temp:
    print(key, temp[key].shape if torch.is_tensor(temp[key]) else type(temp[key]))